<a href="https://colab.research.google.com/github/SiriRRR/590-project/blob/main/mt5-fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q transformers torch
!pip install datasets

!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install rouge_score
!pip install evaluate

In [29]:
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict

from datasets import load_dataset
import pandas as pd
import torch
from transformers.tokenization_utils_base import BatchEncoding
from typing import List, Tuple
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric
import tensorflow as tf
import os
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from nltk.tokenize import sent_tokenize
import nltk
import evaluate
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
ds_train, ds_val, ds_test = load_dataset("cnn_dailymail",'3.0.0',split= ['train[:1%]', 'validation[:1%]','test[:1%]'])
ds_train

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2871
})

In [4]:
ds_val

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 134
})

In [5]:
ds_test

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 115
})

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [8]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!huggingface-cli login

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [12]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_val =ds_val.map(preprocess_function, batched=True)
tokenized_test =ds_test.map(preprocess_function, batched=True)


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
tokenized_train=tokenized_train.remove_columns(ds_train.column_names)
tokenized_test=tokenized_test.remove_columns(ds_test.column_names)
tokenized_val=tokenized_val.remove_columns(ds_val.column_names)

features = [tokenized_train[i] for i in range(2)]

In [15]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_train,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8)
  
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_val,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8)
  

num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01)


model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [16]:
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-test", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=8
)

Cloning https://huggingface.co/SiriRRR/mt5-small-finetuned-test into local empty directory.


Epoch 1/8
358/358 [==============================] - ETA: 0s - loss: 9.7242

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


358/358 [==============================] - 89s 183ms/step - loss: 9.7242 - val_loss: 4.3057
Epoch 2/8
358/358 [==============================] - 37s 102ms/step - loss: 6.0198 - val_loss: 3.7094
Epoch 3/8
358/358 [==============================] - 37s 102ms/step - loss: 5.3389 - val_loss: 3.4579
Epoch 4/8
358/358 [==============================] - 37s 102ms/step - loss: 4.9911 - val_loss: 3.3501
Epoch 5/8
358/358 [==============================] - 37s 102ms/step - loss: 4.7868 - val_loss: 3.2982
Epoch 6/8
358/358 [==============================] - 36s 102ms/step - loss: 4.6160 - val_loss: 3.2603
Epoch 7/8
358/358 [==============================] - 37s 103ms/step - loss: 4.5099 - val_loss: 3.2444
Epoch 8/8
358/358 [==============================] - 37s 102ms/step - loss: 4.4578 - val_loss: 3.2388


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.33k/2.07G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/SiriRRR/mt5-small-finetuned-test
   edf6262..a2106f5  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/SiriRRR/mt5-small-finetuned-test
   edf6262..a2106f5  main -> main



In [17]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_val,
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [18]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [23]:
all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 16/16 [00:01<00:00, 11.08it/s]


In [33]:
rouge_score = evaluate.load("rouge")
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value* 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 16.2172, 'rouge2': 4.9937, 'rougeL': 14.3095, 'rougeLsum': 15.0862}

In [35]:
# use the fine-tune model
#from transformers import pipeline

#hub_model_id = 'SiriRRR/mt5-small-finetuned-test'
#summarizer = pipeline("summarization", model=hub_model_id)

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at SiriRRR/mt5-small-finetuned-test.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/74.0 [00:00<?, ?B/s]